This project is about analyzing the causes of fires in California, particularly mand-made vs. natural, to attempt to predict when fires might be likelier to occur. This project will be analyzing the scale and location of the fires, comparing those factors to the cause, and attempt to find trends that might indicate when/where a fire is likelier to break out.

[insert rationale for why certain data sets were used and how they were prepared after actually doing the work below]
There are two datasets strictly about fires that are going to be looked at. The first is a record of all fires dating back from 1878, although this will be clipped to 1950 to present to match the data available for lightning occurences in California. Both of these datasets have location data, so we can compare yearly fires and thunderstorm occurences for specific regions. This allows us to understand how much of a threat, in terms of fire likelihood, lightning can be. Additionally, there are two more data sets that analyze small(<5000 acres) and large(>=5000 acres) fires over the past five years, so we can focus on both very broad trends over many years, as well as more recent trends that might better help us predict the immediate future.